# Analysis of Public Education in the US
***
Patricia Carcamo Acosta
February 19, 2020
***


### 1. Initiate Spark Environment
***

In [1]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

### 2. Data Source and DataFrame Setup
***
The main dataset that will be used for the analysis is districtDF. The secondary dataset, stateDF, will enrich our analysis with reading and math performance scores for each state. 
I generated the following additional features which are pertinent to this analysis: 
1. Federal Ratio - the percentage of federal revenue as a part of total revenue
2. FedEnrollmentAllotment - the persentage of federal revenue to the number of students enrolled
3. PerStudentExp - the percentage of total expenditure per number of students enrolled

In [2]:
stateDF = spark.read\
                    .option("inferSchema", "true") \
                    .option("header", "true") \
                    .csv("states_all_extended.csv")

disDF = spark.read\
                    .option("inferSchema", "true") \
                    .option("header", "true") \
                    .csv("finance_districts.csv")

districtDF = disDF.where("TOTALREV > 0 and TFEDREV >0 and TLOCREV > 0 and TSTREV > 0")\
                          .withColumn("FedRatio", disDF.TFEDREV/disDF.TOTALREV)\
                          .withColumn("FedEnrollmentAllotment", disDF.TFEDREV/disDF.ENROLL)\
                          .withColumn("PerStudentExp", disDF.TOTALEXP/disDF.ENROLL)\
                          .withColumn("PercInstruction", disDF.TCURINST/disDF.TOTALEXP)\
                          .withColumn("PercSpecServices", disDF.TCURSSVC/disDF.TOTALEXP)\
                          .cache()

### 3. Display DataFrame Size and Schema
***
districsDF has 361,030 distinct rows, stateDF has 1,918 distinct rows

In [3]:
from IPython.display import display, Markdown

stateDF.printSchema()
districtDF.printSchema()

distinctState = stateDF.distinct().count()
distinctDistrict = districtDF.distinct().count()

print("There are", distinctState, "distinct rows for State Data")
print("There are", distinctDistrict, "distinct rows for District Data")


root
 |-- PRIMARY_KEY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- ENROLL: double (nullable = true)
 |-- TOTAL_REVENUE: double (nullable = true)
 |-- FEDERAL_REVENUE: double (nullable = true)
 |-- STATE_REVENUE: double (nullable = true)
 |-- LOCAL_REVENUE: double (nullable = true)
 |-- TOTAL_EXPENDITURE: double (nullable = true)
 |-- INSTRUCTION_EXPENDITURE: double (nullable = true)
 |-- SUPPORT_SERVICES_EXPENDITURE: double (nullable = true)
 |-- OTHER_EXPENDITURE: double (nullable = true)
 |-- CAPITAL_OUTLAY_EXPENDITURE: double (nullable = true)
 |-- A_A_A: double (nullable = true)
 |-- G01_A_A: double (nullable = true)
 |-- G02_A_A: double (nullable = true)
 |-- G03_A_A: double (nullable = true)
 |-- G04_A_A: double (nullable = true)
 |-- G05_A_A: double (nullable = true)
 |-- G06_A_A: double (nullable = true)
 |-- G07_A_A: double (nullable = true)
 |-- G08_A_A: double (nullable = true)
 |-- G09_A_A: double (nullable = true)

### 4. Cache DataFrames
***

In [4]:
stateDF.cache()
districtDF.cache()

DataFrame[STATE: string, ENROLL: int, NAME: string, YRDATA: int, TOTALREV: int, TFEDREV: int, TSTREV: int, TLOCREV: int, TOTALEXP: int, TCURINST: int, TCURSSVC: int, TCURONON: int, TCAPOUT: int, FedRatio: double, FedEnrollmentAllotment: double, PerStudentExp: double, PercInstruction: double, PercSpecServices: double]

### 5. Feature Analysis
***

#### State Income
Half of the values in the table stateDF are null. I will extract state income from the districtsDF data instead.

In [5]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit

print("Summary of Revenue and Expenditure Columns: ") 
stateDF.select("TOTAL_REVENUE", "FEDERAL_REVENUE", "STATE_REVENUE", "LOCAL_REVENUE").summary().show()

print("Checking for Nulls in Revenue and Expenditure Columns:")
stateDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["TOTAL_REVENUE", "FEDERAL_REVENUE", "STATE_REVENUE", "LOCAL_REVENUE"]]).show()

print("Checking Distinct Values in Revenue and Expenditure Columns:")
stateDF.select([countDistinct(c).alias(c) for c in ["TOTAL_REVENUE", "FEDERAL_REVENUE", "STATE_REVENUE", "LOCAL_REVENUE"]]).show()


Summary of Revenue and Expenditure Columns: 
+-------+--------------------+------------------+-----------------+------------------+
|summary|       TOTAL_REVENUE|   FEDERAL_REVENUE|    STATE_REVENUE|     LOCAL_REVENUE|
+-------+--------------------+------------------+-----------------+------------------+
|  count|                1275|              1275|             1275|              1275|
|   mean|   9102044.929411765| 767779.8643137255|4223742.902745098|4110522.1560784313|
| stddev|1.1759620227524102E7|1146991.5073249412|5549734.818393522| 5489561.973466697|
|    min|            465650.0|           31020.0|              0.0|           22093.0|
|    25%|           2188587.0|          189689.0|        1165178.0|          714562.0|
|    50%|           5085826.0|          403548.0|        2537754.0|         2058996.0|
|    75%|          1.084872E7|          828432.0|        5066552.0|         4768381.0|
|    max|         8.9217262E7|         9990221.0|      5.0904567E7|       3.6105265E7

#### State Expenses
Again, half of the values in the table stateDF are null. I will extract state expenses from the districtsDF data instead.

In [6]:
print("Summary of Revenue and Expenditure Columns: ") 
stateDF.select("TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE").summary().show()

print("Checking for Nulls in Revenue and Expenditure Columns:")
stateDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]).show()

print("Checking Distinct Values in Revenue and Expenditure Columns:")
stateDF.select([countDistinct(c).alias(c) for c in ["TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]).show()


Summary of Revenue and Expenditure Columns: 
+-------+--------------------+-----------------------+----------------------------+--------------------------+
|summary|   TOTAL_EXPENDITURE|INSTRUCTION_EXPENDITURE|SUPPORT_SERVICES_EXPENDITURE|CAPITAL_OUTLAY_EXPENDITURE|
+-------+--------------------+-----------------------+----------------------------+--------------------------+
|  count|                1275|                   1275|                        1275|                      1275|
|   mean|   9206242.121568628|      4768010.484705882|          2682586.5192156862|         903467.4549019608|
| stddev|1.1992791012445949E7|      6300569.339670264|          3357214.4993502507|        1329473.4683338907|
|    min|            481665.0|               265549.0|                    139963.0|                   12708.0|
|    25%|           2169432.0|              1171313.0|                    637240.0|                  181392.0|
|    50%|           5242672.0|              2658253.0|             

#### Math and Reading Scores Grades 4 and 8
Reading and math scores are only available in stateDF. I will clean out null values and use this information to enrich the analysis.  

In [7]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit

print("Summary of Math and Reading Scores for Grades 4 and 8:")
stateDF.select("AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", "AVG_READING_4_SCORE", "AVG_READING_8_SCORE").summary().show()

print("Checking for Nulls in Math and Reading Scores for Grades 4 and 8")
stateDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", "AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]]).show()
print("Cleaning out Null Values")
stateDF = stateDF.filter(stateDF.AVG_MATH_4_SCORE.isNotNull())
stateDF = stateDF.filter(stateDF.AVG_MATH_8_SCORE.isNotNull())
stateDF = stateDF.filter(stateDF.AVG_READING_4_SCORE.isNotNull())
stateDF = stateDF.filter(stateDF.AVG_READING_8_SCORE.isNotNull())
stateDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", "AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]]).show()


print("Checking for Distinct Values in Math and Reading Scores for Grades 4 and 8")
stateDF.select([countDistinct(c).alias(c) for c in ["AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", "AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]]).show()


Summary of Math and Reading Scores for Grades 4 and 8:
+-------+------------------+------------------+-------------------+-------------------+
|summary|  AVG_MATH_4_SCORE|  AVG_MATH_8_SCORE|AVG_READING_4_SCORE|AVG_READING_8_SCORE|
+-------+------------------+------------------+-------------------+-------------------+
|  count|               535|               531|                532|                497|
|   mean|234.79728009682898|  278.461452552044| 218.89788614753934|  263.6833246527511|
| stddev|10.209001517937478| 10.11188590279621|  7.742281313769915|  6.792513207431028|
|    min|  187.134669958413|  232.831509532896|   178.557611651728| 236.37910176331403|
|    25%|  229.704281935704|272.76180889422903|   214.663401291545|   259.547225161046|
|    50%|237.26482055278802|  280.619538285703| 220.41603448313703|   265.022858505901|
|    75%|  242.009195680162|  285.539188724412|   223.999336825378|   268.197442761373|
|    max|  253.420961133202|  300.568235011147|   236.77386658036

#### State Student Enrollment
There are some missing values in Enrollments in districtsDF, these will be cleaned further down.

In [8]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit

print("Summary of State Enrollment:")
districtDF.select("STATE", "YRDATA", "ENROLL").summary().show()

print("Checking for Nulls in State Enrollment")
districtDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["STATE", "YRDATA", "ENROLL"]]).show()

print("Checking for Distinct Values in State Enrollment")
districtDF.select([countDistinct(c).alias(c) for c in ["STATE", "YRDATA", "ENROLL"]]).show()

Summary of State Enrollment:
+-------+-------+------------------+------------------+
|summary|  STATE|            YRDATA|            ENROLL|
+-------+-------+------------------+------------------+
|  count| 361030|            361030|            346035|
|   mean|   null|2003.9746530759217|3239.6257632898405|
| stddev|   null| 7.142621002252625|14249.101260926955|
|    min|ALABAMA|              1992|                 0|
|    25%|   null|              1998|               323|
|    50%|   null|              2004|              1015|
|    75%|   null|              2010|              2708|
|    max|WYOMING|              2016|           1077381|
+-------+-------+------------------+------------------+

Checking for Nulls in State Enrollment
+-----+------+------+
|STATE|YRDATA|ENROLL|
+-----+------+------+
|    0|     0| 14995|
+-----+------+------+

Checking for Distinct Values in State Enrollment
+-----+------+------+
|STATE|YRDATA|ENROLL|
+-----+------+------+
|   50|    25| 23845|
+-----+----

#### District Revenue Expense and Enrollment

In [9]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit

print("Summary of District Data:")
districtDF.select("ENROLL", "YRDATA", "TOTALREV", "TOTALEXP", "TCURINST", "TCURSSVC").summary().show()

print("Checking for Nulls in District Data:")
districtDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["ENROLL", "YRDATA", "TOTALREV", "TOTALEXP", "TCURINST", "TCURSSVC"]]).show()

print("Cleaning out Null Values")
districtDF = districtDF.filter(districtDF.ENROLL.isNotNull())
districtDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["ENROLL", "YRDATA", "TOTALREV", "TOTALEXP", "TCURINST", "TCURSSVC"]]).show()

print("Checking for Distinct Values in District Data:")
districtDF.select([countDistinct(c).alias(c) for c in ["ENROLL", "YRDATA", "TOTALREV", "TOTALEXP", "TCURINST", "TCURSSVC"]]).show()


Summary of District Data:
+-------+------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|            ENROLL|            YRDATA|         TOTALREV|          TOTALEXP|          TCURINST|          TCURSSVC|
+-------+------------------+------------------+-----------------+------------------+------------------+------------------+
|  count|            346035|            361030|           361030|            361030|            361030|            361030|
|   mean|3239.6257632898405|2003.9746530759217|32019.87706007811| 32386.08861867435|16788.271705952415| 9431.767285267153|
| stddev|14249.101260926955| 7.142621002252625|181476.4188982236|193529.79490904833| 118505.9501969893|43791.991645684706|
|    min|                 0|              1992|                6|                 0|                 0|                 0|
|    25%|               323|              1998|             3562|              3492|              1751|          

### Business Analysis

#### What states have the highest and lowest overall revenue, and highest state/local revenue compared to federal. What schools have a high percentage of federal revenue to total revenue?



In [10]:
stateRevDF = districtDF.select("STATE","TOTALREV", "TFEDREV", "TSTREV", "TLOCREV").groupBy("STATE")\
                            .agg(
                            F.sum("TOTALREV").alias("TotalRevenue"),
                            F.sum("TFEDREV").alias("FederalRevenue"),
                            F.sum("TSTREV").alias("StateRevenue"),
                            F.sum("TLOCREV").alias("LocalRevenue"),
                            F.mean("TOTALREV").alias("AverageTotalRevenue")
                                ).orderBy("TotalRevenue", ascending=False)

stateRevDF.show()

stateRev2016DF = districtDF.filter(districtDF.YRDATA == 2016)\
                        .select("STATE","TOTALREV", "TFEDREV", "TSTREV", "TLOCREV").groupBy("STATE")\
                            .agg(
                            F.sum("TOTALREV").alias("TotalRevenue"),
                            F.sum("TFEDREV").alias("FederalRevenue"),
                            F.sum("TSTREV").alias("StateRevenue"),
                            F.sum("TLOCREV").alias("LocalRevenue"),
                            F.mean("TOTALREV").alias("AverageTotalRevenue")
                                ).orderBy("TotalRevenue", ascending=False)

stateRev2016DF.show()

FedRatio2016 = districtDF.filter(districtDF.YRDATA == 2016).select("STATE", "NAME", "FedRatio").sort(desc("FedRatio")).show(10, False)
FedRatio2016 = districtDF.filter(districtDF.YRDATA == 2016).select("STATE", "NAME", "FedRatio").sort("FedRatio").show(10, False)

+--------------+------------+--------------+------------+------------+-------------------+
|         STATE|TotalRevenue|FederalRevenue|StateRevenue|LocalRevenue|AverageTotalRevenue|
+--------------+------------+--------------+------------+------------+-------------------+
|    CALIFORNIA|  1366095668|     142238910|   757294332|   466562426| 56176.316637881406|
|      NEW_YORK|  1037863682|      63666794|   438065910|   536130978| 63069.013247447736|
|         TEXAS|   904045938|      94330559|   355985292|   453730087|  35959.02859870331|
|  PENNSYLVANIA|   522078436|      37005885|   184440431|   300632120|  36652.51586632968|
|      ILLINOIS|   521524895|      42297401|   176629129|   302598365|  22155.77955733039|
|       FLORIDA|   510121985|      52786336|   213249125|   244086524| 317240.04042288556|
|    NEW_JERSEY|   491576313|      21055855|   191575298|   278945160|  36819.43772002097|
|          OHIO|   443185813|      30885616|   184689998|   227610199| 25871.909690601286|

In [11]:
stateRevDF = districtDF.select("STATE","TOTALREV", "TFEDREV", "TSTREV", "TLOCREV").groupBy("STATE")\
                            .agg(
                            F.sum("TOTALREV").alias("TotalRevenue"),
                            F.sum("TFEDREV").alias("FederalRevenue"),
                            F.sum("TSTREV").alias("StateRevenue"),
                            F.sum("TLOCREV").alias("LocalRevenue"),
                            F.mean("TOTALREV").alias("AverageTotalRevenue")
                                ).orderBy("TotalRevenue", ascending=True)
stateRevDF.show()

stateRev2016DF = districtDF.filter(districtDF.YRDATA == 2016).select("STATE","TOTALREV", "TFEDREV", "TSTREV", "TLOCREV").groupBy("STATE")\
                            .agg(
                            F.sum("TOTALREV").alias("TotalRevenue"),
                            F.sum("TFEDREV").alias("FederalRevenue"),
                            F.sum("TSTREV").alias("StateRevenue"),
                            F.sum("TLOCREV").alias("LocalRevenue"),
                            F.mean("TOTALREV").alias("AverageTotalRevenue")
                                ).orderBy("TotalRevenue", ascending=True).show(10)

FedRatio2016 = districtDF.filter(districtDF.YRDATA == 2016).select("STATE", "NAME", "FedRatio").groupBy("STATE").agg(F.mean("FedRatio").alias("FedRatio")).sort(desc("FedRatio")).show(10, False)
FedRatio2016 = districtDF.filter(districtDF.YRDATA == 2016).select("STATE", "NAME", "FedRatio").groupBy("STATE").agg(F.mean("FedRatio").alias("FedRatio")).sort("FedRatio").show(10, False)

+-------------+------------+--------------+------------+------------+-------------------+
|        STATE|TotalRevenue|FederalRevenue|StateRevenue|LocalRevenue|AverageTotalRevenue|
+-------------+------------+--------------+------------+------------+-------------------+
| NORTH_DAKOTA|    24334208|       2890473|    10587395|    10856340| 4483.9152386217065|
| SOUTH_DAKOTA|    24502988|       3504065|     7852643|    13146280| 6490.8577483443705|
|      WYOMING|    28719069|       2132311|    14972263|    11614495| 24822.012964563528|
|      VERMONT|    31514799|       1973220|    18954348|    10587231|  5029.492339610597|
|      MONTANA|    31571370|       3956335|    14786574|    12828461| 3027.8478948882707|
|     DELAWARE|    33278940|       2448065|    20332815|    10498060|  73140.52747252748|
|        IDAHO|    40838263|       4257546|    25284343|    11296374| 15181.510408921933|
|       ALASKA|    41948649|       6180482|    26022251|     9745916|  32900.90117647059|
| RHODE_IS

#### What was the revenue distribution for the school districts? Who has the highest revenue overall, and in 2016? 

In [12]:
districtRevDF = districtDF.select("STATE","YRDATA","NAME","TOTALREV", "TFEDREV", "TSTREV", "TLOCREV")\
                            .orderBy("TOTALREV", ascending=False)
districtRevDF.show(10, False)

districtRevDF = districtDF.select("NAME", "TOTALREV").groupBy("NAME").agg(F.sum("TOTALREV").alias("OverallRevenue")).sort(desc("OverallRevenue")).show(10, False)

districtRevDF = districtDF.filter(districtDF.YRDATA == 2016).select("NAME", "TOTALREV").groupBy("NAME").agg(F.sum("TOTALREV").alias("Revenue2016")).sort(desc("Revenue2016")).show(10, False)

+--------+------+-----------------------------+--------+-------+--------+--------+
|STATE   |YRDATA|NAME                         |TOTALREV|TFEDREV|TSTREV  |TLOCREV |
+--------+------+-----------------------------+--------+-------+--------+--------+
|NEW_YORK|2016  |NEW YORK CITY SCHOOL DISTRICT|27448356|1739101|10568010|15141245|
|NEW_YORK|2015  |NEW YORK CITY SCHOOL DISTRICT|25437384|1307783|9837509 |14292092|
|NEW_YORK|2014  |NEW YORK CITY SCHOOL DISTRICT|24215438|1814405|9352527 |13048506|
|NEW_YORK|2012  |NEW YORK CITY SCHOOL DISTRICT|23517452|2237047|8648188 |12632217|
|NEW_YORK|2013  |NEW YORK CITY SCHOOL DISTRICT|23438719|1780098|8635858 |13022763|
|NEW_YORK|2011  |NEW YORK CITY SCHOOL DISTRICT|22651559|3120314|8594973 |10936272|
|NEW_YORK|2009  |NEW YORK CITY SCHOOL DISTRICT|21165693|1817770|10040718|9307205 |
|NEW_YORK|2010  |NEW YORK CITY SCHOOL DISTRICT|21023695|2047926|8375172 |10600597|
|NEW_YORK|2008  |NEW YORK CITY SCHOOL DISTRICT|19885693|1785145|9335189 |8765359 |
|NEW

In [13]:
districtRevDF = districtDF.select("STATE","NAME","TOTALREV", "TFEDREV", "TSTREV", "TLOCREV") \
                    .filter(districtDF.TOTALREV > 10000) \
                    .orderBy("TOTALREV", ascending=True).show(20, False)

+-------------+-----------------------------------+--------+-------+------+-------+
|STATE        |NAME                               |TOTALREV|TFEDREV|TSTREV|TLOCREV|
+-------------+-----------------------------------+--------+-------+------+-------+
|MINNESOTA    |BRAHAM SCH DIST 314                |10001   |511    |7107  |2383   |
|NEW_JERSEY   |HENRY HUDSON REGIONAL SCH DIST     |10001   |312    |1468  |8221   |
|OHIO         |ST HENRY LOCAL SCH DIST            |10001   |381    |6055  |3565   |
|KENTUCKY     |TODD CO SCH DIST                   |10001   |1039   |7520  |1442   |
|NEW_HAMPSHIRE|HOPKINTON SCH DIST                 |10001   |426    |4058  |5517   |
|MASSACHUSETTS|STURBRIDGE TOWN SCHS               |10001   |548    |2227  |7226   |
|ARKANSAS     |SOUTHEAST ARKANSAS EDUC SERV COOP  |10001   |3223   |3691  |3087   |
|CALIFORNIA   |PINER OLIVET UNION ELEM SCHOOL DIST|10001   |266    |5809  |3926   |
|IDAHO        |PRESTON JT SCH DIST 201            |10001   |552    |7960  |1

In [14]:
FedRatioDF = districtDF.select("STATE", "NAME", "FedRatio").orderBy("FedRatio", ascending=False).show(20, False)

+------------+---------------------------------------------+------------------+
|STATE       |NAME                                         |FedRatio          |
+------------+---------------------------------------------+------------------+
|CALIFORNIA  |CENTRAL CALIFORNIA MIGRANT HEAD START JPA    |0.9980877347305618|
|ILLINOIS    |MACKINAW VALLEY SPECIAL EDUCATION ASSOCIATION|0.9955706984667803|
|ILLINOIS    |FEDERATION OF DISTRICTS FOR SPECIAL EDUCATION|0.9824591573516767|
|ILLINOIS    |FEDERATION OF DISTRICTS FOR SPECIAL EDUCATION|0.9809504509068802|
|ILLINOIS    |FEDERATION OF DISTRICTS FOR SPECIAL EDUCATION|0.9741654571843251|
|ILLINOIS    |FEDERATION OF DISTRICTS FOR SPECIAL EDUCATION|0.9721898417985012|
|NORTH_DAKOTA|MINOT AFB 160 SCHOOL DISTRICT                |0.9719519175729823|
|CALIFORNIA  |CONTRA COSTA CO ROP                          |0.9707639328132687|
|ILLINOIS    |FED OF DISTRICTS FOR SPECIAL EDUC            |0.9680200658410409|
|ILLINOIS    |FED OF DISTRICTS FOR SPECI

#### What states and districts have the highest student enrollment totals in 2016? 

In [15]:
FedEnrollAllotDF = districtDF.filter(districtDF.YRDATA == 2016).select("STATE", "ENROLL").groupBy("STATE")\
                             .agg(F.sum("ENROLL").alias("TotalEnrollment")).sort(desc("TotalEnrollment")).show(10)

FedEnrollAllotDF = districtDF.filter(districtDF.YRDATA == 2016).select("STATE","NAME", "ENROLL").groupBy("NAME")\
                             .agg(F.sum("ENROLL").alias("TotalEnrollment")).sort(desc("TotalEnrollment")).show(10, False)

+--------------+---------------+
|         STATE|TotalEnrollment|
+--------------+---------------+
|    CALIFORNIA|        6217031|
|         TEXAS|        5053260|
|       FLORIDA|        2776933|
|      NEW_YORK|        2590921|
|      ILLINOIS|        2030717|
|       GEORGIA|        1727085|
|          OHIO|        1595024|
|  PENNSYLVANIA|        1572587|
|NORTH_CAROLINA|        1462036|
|    NEW_JERSEY|        1364266|
+--------------+---------------+
only showing top 10 rows

+----------------------------------------+---------------+
|NAME                                    |TotalEnrollment|
+----------------------------------------+---------------+
|NEW YORK CITY SCHOOL DISTRICT           |981667         |
|LOS ANGELES UNIF SCH DIST               |639337         |
|CITY OF CHICAGO SCHOOL DISTRICT 299     |387311         |
|MIAMI-DADE COUNTY PUBLIC SCHOOL DISTRICT|357579         |
|CLARK COUNTY SCHOOL DISTRICT            |325990         |
|BROWARD COUNTY SCHOOL DISTRICT         

#### Calculate State Expenditures and what percentage of total goes to instruction and services. Identify the states that spend most and least on instruction and services. Analyze who spends the most and the least per student. Calculate the average spenditure per student for the US. 


In [16]:
stateExpDF = districtDF.select("STATE","ENROLL","TOTALEXP", "TCURINST", "TCURSSVC", "TCAPOUT", "PerStudentExp", "PercInstruction", "PercSpecServices").groupBy("STATE")\
                            .agg(
                            F.round(F.sum("TOTALEXP"),2).alias("AvgTotalExpense"),
                            F.round(F.sum("TCURINST"),2).alias("AvgInstrExpense"),
                            F.round(F.sum("TCURSSVC"),2).alias("AvgServicesExpense"),
                            F.round(F.sum("TCAPOUT"),2).alias("AvgCapitalExpense"),
                            F.round(F.sum("PerStudentExp"),2).alias("AvgPerStudent"),
                            F.round(F.mean("PercInstruction"),2).alias("PercInstruction"),
                            F.round(F.mean("PercSpecServices"),2).alias("PercSpecServ"))\
                            .sort(desc("AvgTotalExpense"))

stateExpDF.show(10)

stateExp2016DF = districtDF.filter(districtDF.YRDATA == 2016).select("STATE","ENROLL","TOTALEXP", "TCURINST", "TCURSSVC", "TCAPOUT", "PerStudentExp").groupBy("STATE")\
                            .agg(
                            F.round(F.sum("TOTALEXP"),2).alias("AvgTotalExpense"),
                            F.round(F.sum("TCURINST"),2).alias("AvgInstructionExpense"),
                            F.round(F.sum("TCURSSVC"),2).alias("AvgServicesExpense"),
                            F.round(F.sum("TCAPOUT"),2).alias("AvgCapitalExpense"),
                            F.round(F.sum("PerStudentExp"),2).alias("AveragePerStudentExpense")
                                ).sort(desc("AvgTotalExpense"))

stateExp2016DF.show(10)

+------------+---------------+---------------+------------------+-----------------+-------------+---------------+------------+
|       STATE|AvgTotalExpense|AvgInstrExpense|AvgServicesExpense|AvgCapitalExpense|AvgPerStudent|PercInstruction|PercSpecServ|
+------------+---------------+---------------+------------------+-----------------+-------------+---------------+------------+
|  CALIFORNIA|     1376348636|      681838483|         396297306|        151778171|    313419.88|           0.52|         0.3|
|    NEW_YORK|     1082069634|      663248625|         265722574|         90495880|    282617.47|           0.58|        0.29|
|       TEXAS|      940855055|      448147169|         255461324|        126933872|    263524.97|            0.5|        0.29|
|PENNSYLVANIA|      532204915|      266449424|         147620479|         42783505|    138007.76|           0.51|        0.29|
|    ILLINOIS|      525742358|      266227306|         164544173|         47859027|    207314.97|           0.4

In [22]:
districtExpDF = districtDF.select("NAME", "TOTALEXP", "TCURINST", "TCURSSVC", "TCAPOUT").groupBy("NAME")\
                .agg(
                F.round(F.avg("TOTALEXP"),2).alias("AvgTotalExpense"),
                            F.round(F.avg("TCURINST"),2).alias("AvgInstructionExpense"),
                            F.round(F.avg("TCURSSVC"),2).alias("AvgServicesExpense"),
                            F.round(F.avg("TCAPOUT"),2).alias("AvgCapitalExpense")
                ).orderBy("AvgTotalExpense", ascending=False)
    
districtExpDF.show(10, False)

+----------------------------------------+---------------+---------------------+------------------+-----------------+
|NAME                                    |AvgTotalExpense|AvgInstructionExpense|AvgServicesExpense|AvgCapitalExpense|
+----------------------------------------+---------------+---------------------+------------------+-----------------+
|NEW YORK CITY SCHOOL DISTRICT           |2.181980627E7  |1.469525153E7        |3843571.4         |2138993.67       |
|NEW YORK CITY SCH DIST                  |1.070267044E7  |6269854.0            |2129922.89        |1236391.0        |
|LOS ANGELES UNIF SCH DIST               |7273471.13     |3604382.5            |2079924.29        |912262.38        |
|CITY OF CHICAGO SCHOOL DISTRICT 299     |4953948.06     |2644138.94           |1370217.5         |468742.83        |
|MIAMI-DADE COUNTY PUBLIC SCHOOL DISTRICT|3818065.3      |2016163.4            |1028034.4         |395116.6         |
|MIAMI-DADE COUNTY PUBLIC SCHOOL DIST    |3220725.5     

#### What states and districts spend the highest percentage of their budget in instruction and Support Services?

In [23]:
districtDF.select(F.mean("PercInstruction")).show(1, False)

districtDF.select("STATE", "NAME", "PercInstruction").groupBy("STATE")\
        .agg(F.mean("PercInstruction").alias("Percent Instruction Expense"))\
        .sort(desc("Percent Instruction Expense")).show(10, False)

districtExpDF.select("NAME", "AvgInstructionExpense").groupBy("NAME")\
    .agg(
        F.sum("AvgInstructionExpense").alias("Highest Average Instruction Expense")
        ).sort(desc("Highest Average Instruction Expense")).show(10, False)

districtDF.filter(districtDF.YRDATA == 2016).select("STATE", "NAME", "PercInstruction").groupBy("NAME", "STATE")\
        .agg(F.mean("PercInstruction").alias("Percent Instruction Expense"))\
        .sort(desc("Percent Instruction Expense")).show(10, False)

districtDF.filter(districtDF.YRDATA == 2016).filter(districtDF.PercInstruction > 0.1).select("STATE", "NAME", "PercInstruction").groupBy("NAME", "STATE")\
        .agg(F.mean("PercInstruction").alias("Percent Instruction Expense"))\
        .sort("Percent Instruction Expense").show(10, False)

+--------------------+
|avg(PercInstruction)|
+--------------------+
|0.5165409439394407  |
+--------------------+

+--------------+---------------------------+
|STATE         |Percent Instruction Expense|
+--------------+---------------------------+
|RHODE_ISLAND  |0.588857423421656          |
|NEBRASKA      |0.5879001009852706         |
|NEW_YORK      |0.5752419141377827         |
|TENNESSEE     |0.5708816553287256         |
|MASSACHUSETTS |0.5665089098331202         |
|NORTH_CAROLINA|0.5559114674732569         |
|HAWAII        |0.5540853294269285         |
|CONNECTICUT   |0.5537003664530593         |
|MONTANA       |0.5509676509562215         |
|IDAHO         |0.5473677163133401         |
+--------------+---------------------------+
only showing top 10 rows

+----------------------------------------+-----------------------------------+
|NAME                                    |Highest Average Instruction Expense|
+----------------------------------------+----------------------------

In [25]:
districtDF.select(F.mean("PercSpecServices")).show(1, False)

districtDF.select("STATE", "NAME", "PercSpecServices").groupBy("STATE")\
        .agg(F.mean("PercSpecServices").alias("Percent Support Expense"))\
        .sort(desc("Percent Support Expense")).show(10, False)

districtExpDF.select("NAME", "AvgServicesExpense").groupBy("NAME")\
    .agg(
        F.sum("AvgServicesExpense").alias("Highest Sup Services Expense")
        ).sort(desc("Highest Sup Services Expense")).show(10, False)

districtDF.filter(districtDF.YRDATA == 2016).select("STATE", "NAME", "PercSpecServices").groupBy("NAME", "STATE")\
        .agg(F.mean("PercSpecServices").alias("Percent Support Expense"))\
        .sort(desc("Percent Support Expense")).show(10, False)

districtDF.filter(districtDF.YRDATA == 2016).filter(districtDF.PercSpecServices > 0.1).select("STATE", "NAME", "PercSpecServices").groupBy("NAME", "STATE")\
        .agg(F.mean("PercSpecServices").alias("Percent Support Expense"))\
        .sort("Percent Support Expense").show(10, False)

+---------------------+
|avg(PercSpecServices)|
+---------------------+
|0.3058507599506189   |
+---------------------+

+----------+-----------------------+
|STATE     |Percent Support Expense|
+----------+-----------------------+
|ALASKA    |0.36049291309291925    |
|OREGON    |0.35702903889348203    |
|NEW_MEXICO|0.35209838405745675    |
|OHIO      |0.3465931562213563     |
|ARIZONA   |0.3376417216600479     |
|MONTANA   |0.33757141410293806    |
|NEVADA    |0.3363350162335723     |
|COLORADO  |0.3347015694711316     |
|ILLINOIS  |0.3297833661557004     |
|MICHIGAN  |0.32823647849115756    |
+----------+-----------------------+
only showing top 10 rows

+----------------------------------------+----------------------------+
|NAME                                    |Highest Sup Services Expense|
+----------------------------------------+----------------------------+
|NEW YORK CITY SCHOOL DISTRICT           |3843571.4                   |
|NEW YORK CITY SCH DIST                  |21299

### Join Revenue and Expense Data with State Reading, Math Scores 
To further the analysis, I will be joining the Revenue and Expense data with State Reading and Math Scores. 

In [27]:
stateReadMath = stateDF.select("STATE", "AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", "AVG_READING_4_SCORE", "AVG_READING_8_SCORE")
stateRevExp = stateRevDF.join(stateExpDF, on="STATE", how="inner")
schoolsDF = stateReadMath.join(stateRevExp, on = "STATE", how = "inner")

print("Let's look at the structure of the joint data frames")
stateReadMath.printSchema()
stateRevExp.printSchema()
schoolsDF.printSchema()

Let's look at the structure of the joint data frames
root
 |-- STATE: string (nullable = true)
 |-- AVG_MATH_4_SCORE: double (nullable = true)
 |-- AVG_MATH_8_SCORE: double (nullable = true)
 |-- AVG_READING_4_SCORE: double (nullable = true)
 |-- AVG_READING_8_SCORE: double (nullable = true)

root
 |-- STATE: string (nullable = true)
 |-- TotalRevenue: long (nullable = true)
 |-- FederalRevenue: long (nullable = true)
 |-- StateRevenue: long (nullable = true)
 |-- LocalRevenue: long (nullable = true)
 |-- AverageTotalRevenue: double (nullable = true)
 |-- AvgTotalExpense: long (nullable = true)
 |-- AvgInstrExpense: long (nullable = true)
 |-- AvgServicesExpense: long (nullable = true)
 |-- AvgCapitalExpense: long (nullable = true)
 |-- AvgPerStudent: double (nullable = true)
 |-- PercInstruction: double (nullable = true)
 |-- PercSpecServ: double (nullable = true)

root
 |-- STATE: string (nullable = true)
 |-- AVG_MATH_4_SCORE: double (nullable = true)
 |-- AVG_MATH_8_SCORE: double (

#### Analyze Change in Reading and Math Scores from grade 4 to 8. What states have the highest scores for reading and math. Compare reading and math scores to total expense and to percentage expense in instruction and support services. 


In [28]:
schoolsDF = schoolsDF.withColumn("ChangeReading", schoolsDF.AVG_READING_8_SCORE - schoolsDF.AVG_READING_4_SCORE)\
         .withColumn("ChangeMath", schoolsDF.AVG_MATH_8_SCORE - schoolsDF.AVG_MATH_4_SCORE)

schoolsDF.select(
    F.round(F.mean("AVG_READING_4_SCORE"),0).alias("US Average Reading4"), 
    F.round(F.mean("AVG_READING_8_SCORE"),0).alias("US Average Reading8"), 
    F.round(F.mean("AVG_MATH_4_SCORE"),0).alias("US Average Math4"), 
    F.round(F.mean("AVG_MATH_8_SCORE"),0).alias("US Average Math8"),
    F.round(F.mean("ChangeReading"),0).alias("US Average Change in Reading"),
    F.round(F.mean("ChangeMath"),0).alias("US Average Change Math")
).show()


+-------------------+-------------------+----------------+----------------+----------------------------+----------------------+
|US Average Reading4|US Average Reading8|US Average Math4|US Average Math8|US Average Change in Reading|US Average Change Math|
+-------------------+-------------------+----------------+----------------+----------------------------+----------------------+
|              220.0|              264.0|           237.0|           280.0|                        44.0|                  43.0|
+-------------------+-------------------+----------------+----------------+----------------------------+----------------------+



In [29]:
schoolsDF.printSchema()

reading = schoolsDF.select("STATE", "AVG_READING_4_SCORE", "AVG_READING_8_SCORE", "AvgServicesExpense", "AvgPerStudent", "PercInstruction", "PercSpecServ")\
        .groupBy("STATE")\
        .agg(
        F.round(F.max("AVG_READING_4_SCORE"),0).alias("TopReading4"),
        F.round(F.max("AVG_READING_8_SCORE"),0).alias("TopReading8"),
        F.round(F.sum("AvgServicesExpense"),0).alias("SuppServicesExpense"),
        F.round(F.sum("AvgPerStudent"),0).alias("PerStudentExpense"),
        F.round(F.avg("PercInstruction"),4).alias("PercInstruction"),
        F.round(F.avg("PercSpecServ"),4).alias("PercSpecServices"))\
        .sort(desc("TopReading4")).sort(desc("TopReading8")).show(10, False)


root
 |-- STATE: string (nullable = true)
 |-- AVG_MATH_4_SCORE: double (nullable = true)
 |-- AVG_MATH_8_SCORE: double (nullable = true)
 |-- AVG_READING_4_SCORE: double (nullable = true)
 |-- AVG_READING_8_SCORE: double (nullable = true)
 |-- TotalRevenue: long (nullable = true)
 |-- FederalRevenue: long (nullable = true)
 |-- StateRevenue: long (nullable = true)
 |-- LocalRevenue: long (nullable = true)
 |-- AverageTotalRevenue: double (nullable = true)
 |-- AvgTotalExpense: long (nullable = true)
 |-- AvgInstrExpense: long (nullable = true)
 |-- AvgServicesExpense: long (nullable = true)
 |-- AvgCapitalExpense: long (nullable = true)
 |-- AvgPerStudent: double (nullable = true)
 |-- PercInstruction: double (nullable = true)
 |-- PercSpecServ: double (nullable = true)
 |-- ChangeReading: double (nullable = true)
 |-- ChangeMath: double (nullable = true)

+--------------+-----------+-----------+-------------------+-----------------+---------------+----------------+
|STATE         |To

In [30]:
math = schoolsDF.select("STATE", "AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE", "AvgServicesExpense", "AvgPerStudent", "PercInstruction", "PercSpecServ").groupBy("STATE")\
    .agg(
        F.round(F.max("AVG_MATH_4_SCORE"),0).alias("TopMath4"),
        F.round(F.max("AVG_MATH_8_SCORE"),0).alias("TopMath8"),
        F.round(F.sum("AvgServicesExpense"),0).alias("SuppServicesExpense"),
        F.round(F.sum("AvgPerStudent"),0).alias("PerStudentExpense"),
        F.round(F.avg("PercInstruction"),4).alias("PercInstruction"),
        F.round(F.avg("PercSpecServ"),4).alias("PercSpecServices")
        ).sort(desc("TopMath4")).sort(desc("TopMath8")).show(10, False)

+-------------+--------+--------+-------------------+-----------------+---------------+----------------+
|STATE        |TopMath4|TopMath8|SuppServicesExpense|PerStudentExpense|PercInstruction|PercSpecServices|
+-------------+--------+--------+-------------------+-----------------+---------------+----------------+
|MASSACHUSETTS|253.0   |301.0   |838955860          |1055150.0        |0.57           |0.3             |
|NEW_HAMPSHIRE|253.0   |296.0   |119753208          |422466.0         |0.49           |0.28            |
|NEW_JERSEY   |249.0   |296.0   |1283520480         |1874874.0        |0.51           |0.32            |
|VERMONT      |248.0   |295.0   |75361680           |689418.0         |0.42           |0.29            |
|MINNESOTA    |253.0   |295.0   |468862362          |812058.0         |0.52           |0.25            |
|NORTH_DAKOTA |246.0   |293.0   |58940937           |495406.0         |0.47           |0.29            |
|MONTANA      |244.0   |293.0   |100346960          |11

#### Dig deep: look at the state with best reading and math scores and see revenue and expense distribution by district

In [31]:
MassachusetsDF = districtDF.filter(districtDF.STATE == "MASSACHUSETTS")\
            .select("NAME", "TOTALEXP", "TCURINST", "TCURSSVC", "TCAPOUT", "PercInstruction", "PercSpecServices")\
            .groupBy("NAME")\
            .agg(
                F.sum("TOTALEXP").alias("AvgTotalExp"),
                F.sum("TCURINST").alias("AvgInstrExpense"),
                F.sum("TCURSSVC").alias("AvgSupServExpense"),
                F.sum("TCAPOUT").alias("AvgCapOutExpense"),
                F.round(F.avg("PercInstruction"),2).alias("PercInstr"),
                F.round(F.avg("PercSpecServices"),2).alias("PercSpecServ")
                ).orderBy("AvgTotalExp", ascending=False).show(10)

Massachusets2016DF = districtDF.filter(districtDF.STATE == "MASSACHUSETTS").filter(districtDF.YRDATA == 2016)\
            .select("NAME", "TOTALEXP", "TCURINST", "TCURSSVC", "TCAPOUT", "PercInstruction", "PercSpecServices")\
            .groupBy("NAME")\
            .agg(
                F.sum("TOTALEXP").alias("AvgTotalExp"),
                F.sum("TCURINST").alias("AvgInstrExpense"),
                F.sum("TCURSSVC").alias("AvgSupServExpense"),
                F.sum("TCAPOUT").alias("AvgCapOutExpense"),
                F.round(F.avg("PercInstruction"),2).alias("PercInstr"),
                F.round(F.avg("PercSpecServices"),2).alias("PercSpecServ"))\
            .orderBy("AvgTotalExp", ascending=False).show(10)

+--------------------+-----------+---------------+-----------------+----------------+---------+------------+
|                NAME|AvgTotalExp|AvgInstrExpense|AvgSupServExpense|AvgCapOutExpense|PercInstr|PercSpecServ|
+--------------------+-----------+---------------+-----------------+----------------+---------+------------+
|     BOSTON CITY SCH|   24186218|       13958846|          7844158|          946697|     0.58|        0.32|
|SPRINGFIELD CITY SCH|    8762936|        5027023|          2466008|          528090|     0.58|        0.28|
|WORCESTER PUBLIC ...|    7748038|        4827879|          1912710|          350699|     0.62|        0.25|
|     NEWTON CITY SCH|    4490033|        2575348|          1252801|          479138|     0.59|        0.28|
|   BROCKTON CITY SCH|    4486306|        2638397|          1516341|           96194|     0.59|        0.34|
|     LOWELL CITY SCH|    4430284|        2758023|          1165594|          154056|     0.61|        0.26|
|       LYNN CITY S

In [32]:
FloridaDF = districtDF.filter(districtDF.STATE == "FLORIDA")\
            .select("NAME", "TOTALEXP", "TCURINST", "TCURSSVC", "TCAPOUT", "PercInstruction", "PercSpecServices")\
            .groupBy("NAME")\
            .agg(
                F.sum("TOTALEXP").alias("AvgTotalExp"),
                F.sum("TCURINST").alias("AvgInstrExpense"),
                F.sum("TCURSSVC").alias("AvgSupServExpense"),
                F.sum("TCAPOUT").alias("AvgCapOutExpense"),
                F.round(F.avg("PercInstruction"),2).alias("PercInstr"),
                F.round(F.avg("PercSpecServices"),2).alias("PercSpecServ")
                ).orderBy("AvgTotalExp", ascending=False).show(10)

Florida2016DF = districtDF.filter(districtDF.STATE == "FLORIDA").filter(districtDF.YRDATA == 2016)\
            .select("NAME", "TOTALEXP", "TCURINST", "TCURSSVC", "TCAPOUT", "PercInstruction", "PercSpecServices")\
            .groupBy("NAME")\
            .agg(
                F.sum("TOTALEXP").alias("AvgTotalExp"),
                F.sum("TCURINST").alias("AvgInstrExpense"),
                F.sum("TCURSSVC").alias("AvgSupServExpense"),
                F.sum("TCAPOUT").alias("AvgCapOutExpense"),
                F.round(F.avg("PercInstruction"),2).alias("PercInstr"),
                F.round(F.avg("PercSpecServices"),2).alias("PercSpecServ"))\
            .orderBy("AvgTotalExp", ascending=False).show(10)

+--------------------+-----------+---------------+-----------------+----------------+---------+------------+
|                NAME|AvgTotalExp|AvgInstrExpense|AvgSupServExpense|AvgCapOutExpense|PercInstr|PercSpecServ|
+--------------------+-----------+---------------+-----------------+----------------+---------+------------+
|MIAMI-DADE COUNTY...|   38180653|       20161634|         10280344|         3951166|     0.53|        0.27|
|BROWARD CO SCHOOL...|   30848426|       13936836|          9312618|         4880806|     0.45|        0.31|
|PALM BEACH CO SCH...|   22406774|       10041774|          5543082|         4589867|     0.46|        0.25|
|HILLSBOROUGH CO S...|   21125356|        9604272|          5836099|         3200397|     0.45|        0.28|
|BROWARD COUNTY SC...|   20853451|       11339702|          6077139|         1356766|     0.55|        0.29|
|MIAMI-DADE COUNTY...|   19324353|        9689553|          5714656|         2042745|      0.5|         0.3|
| ORANGE CO SCH BOA